In [1]:
from PIL import Image
import sys
import numpy as np
from scipy.misc import imread
import cPickle as pickle
import os
import matplotlib.pyplot as plt
import argparse
from PIL import Image

##### Kepfeldolgozas menete
Eloszor kezzel osszerendeztem a megfelelo parokat

Volt olyan ahol 4 ember kapcsolodott de 3 olyan is ahol csak 1

Fontos tervezoi dontes: toroljem a szuksegteleneket, vagy uj ures kepeket tegyek bele 

    Mindekettot kiprobaltam, de vegul:
    
Torolni kellett a szuksegteleneket

A meglevo kepeket ujrameretezni aranyosan

    majd a nem illo kepeket keigesziteni feher teglalapokkal
    
    es ezeket is ujrameretezni
    
Aztan lehet csak pickleozni

In [110]:
data_path = "/Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB"
basepath = os.path.join(data_path,'DBs/HQf_withPairs')
# target_shape : a target size of the pictures (it will be square)
target_size = 177 
base_size = (4256,2832)
new_image_signal = 'new'
lang_dict = {}

In [135]:
def create_empty_image(path):
    img = Image.new('RGB', (target_size, target_size), (123, 101, 52))
    img.save(path)

def white_bg_square(img):
    "return a white-background-color image having the img in exact center"
    layer = Image.new('RGB', base_size, (255,255,255))
    layer.paste(img, tuple(map(lambda x:(x[0]-x[1])/2, zip(base_size, img.size))))
    return layer

def cut_edges(img):
    return img.crop(((img.size[0]-img.size[1])/2, 0, (img.size[0]-img.size[1])/2 + img.size[1], img.size[1]))
    
def resize_image(image):
    outfile = os.path.splitext(image)[0] + "_" + new_image_signal +".jpeg"
    img = Image.open(image)
    if img.size != base_size:
#         print img.size
        img = white_bg_square(img)
    temp_width = int(float(target_size) / float(img.size[1]) * float(img.size[0]))
    img = img.resize((temp_width, target_size), Image.ANTIALIAS)
    img = cut_edges(img)
    img = img.convert('1')
    img.save(outfile, "jpeg")
    print('SAVED: ' + outfile)
    
def delete_images(signal = 'new'):
    # Delete all signaled files
    for staff in os.listdir(basepath):
        staff_path = os.path.join(basepath,staff)
        if ".DS_Store" in staff_path:
            os.remove(staff_path)
        else:
            for filename in os.listdir(staff_path):
                if ".DS_Store" in filename or signal in filename:
                    os.remove(staff_path + '/' + filename)

In [136]:
def loadimgs(path, from_=0, to=200, n=0):
    X=[]
    y = []
    cat_dict = {}
    lang_dict = {}
    curr_y = n
    for alphabet in os.listdir(path):
        lang_dict[alphabet] = [curr_y,None]
        siblings_path = os.path.join(path,alphabet)
        category_images=[]
        if ".DS_Store" not in siblings_path and int(alphabet) >= from_ and int(alphabet) <= to:
            for person in os.listdir(siblings_path):
                if ('.jpg' in person):
                    resize_image(siblings_path + '/' + person)
#             image_num = len([name for name in os.listdir(siblings_path) if ("new" in name)])
#             print str(image_num) + " " + alphabet
# #             while image_num < 4:
# #                 image_num += 1
# #                 create_empty_image(siblings_path + '/blank_new' + str(image_num) + '.jpg')
# #                 print "CREATE"
            image_num = 0
            for filename in os.listdir(siblings_path):
                if (new_image_signal in filename and image_num < 2):
                    image_path = os.path.join(siblings_path, filename)
                    image = imread(image_path)
#                     print(image.shape)
                    category_images.append(image)
                    y.append(curr_y)
                    image_num +=1
            X.append(np.stack(category_images))
            curr_y += 1
            lang_dict[alphabet][1] = curr_y - 1
            
    y = np.vstack(y)
    X = np.stack(X)
    return X,y,lang_dict

In [137]:
delete_images()
X,y,c=loadimgs(basepath, from_=0, to=200)
with open("trainSib.pickle", "wb") as f:
    pickle.dump((X,c),f, protocol=2)


X,y,c=loadimgs(basepath, from_=201, to=300)
with open("valSib.pickle", "wb") as f:
    pickle.dump((X,c),f)





266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/59/_DSC6001_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/59/_DSC6005_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/59/_DSC6012_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/104/_DSC0195_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/104/_DSC0181_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/57/_DSC5832_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/57/_DSC5835_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/150/_DSC0572_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/1

SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/31/_DSC5905_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/165/_DSC0652_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/165/_DSC0657_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/91/_DSC0099_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/91/_DSC0091_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/62/_DSC9913_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/62/_DSC9905_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/96/_DSC0138_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/96/_DS

SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/228/_DSC2898_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/250/_DSC3023_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/250/_DSC3013_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/266/_DSC3067_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/266/_DSC3088_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/266/_DSC3077_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/204/_DSC0868_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/204/_DSC0875_new.jpeg
266
SAVED: /Users/krisz/Documents/University/Szakdoga/one-shot/SiblingsDB/DBs/HQf_withPairs/